In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score



In [8]:
#We shall begin by importing the 2 US bureau dataframes from each year that we are concerned with:

df1 = pd.read_excel("/Users/jacob/Documents/4YP data/US_BureaOfLaborStatistics/annualstats/national_M2019_dl.xlsx")
df2 = pd.read_excel("/Users/jacob/Documents/4YP data/US_BureaOfLaborStatistics/annualstats/national_M2014_dl.xlsx")


In [9]:
#We shall now drop the unimportant columns:

df1 = df1[["occ_code","occ_title","tot_emp"]]
df1.columns= df1.columns.str.upper()
#df1 = df1.rename(columns={"occ_code": "OCC_CODE", "occ_title": "OCC_TITLE", "tot_emp": "TOT_EMP"}) #ensure consistency of labels
df2 = df2[["OCC_CODE","OCC_TITLE","TOT_EMP"]]


#Now add a holder column for the data to be added to df1:

df1["TOT_EMP2"]=np.nan
df1.head()

,OCC_CODE,OCC_TITLE,TOT_EMP,TOT_EMP2
0,00-0000,All Occupations,146875480,NaN
1,11-0000,Management Occupations,8054120,NaN
2,11-1000,Top Executives,2658440,NaN
3,11-1010,Chief Executives,205890,NaN
4,11-1011,Chief Executives,205890,NaN


In [10]:
#Now we drop duplicate rows on both dataframes:

df1 = df1.drop_duplicates(subset = ["OCC_CODE"],keep="first")
df1 = df1.drop_duplicates(subset = ["TOT_EMP"],keep="first")
df2 = df2.drop_duplicates(subset = ["OCC_CODE"],keep="first")
df2 = df2.drop_duplicates(subset = ["TOT_EMP"],keep="first")

df1.reset_index(drop=True,inplace=True)
df2.reset_index(drop=True,inplace=True)

In [ ]:
#We shall now focus on concatenating these dataframes

def concat_data(a,b,key):
    my_list = list(b["OCC_CODE"])
    for i in range(0,len(my_list)):
        if my_list[i] in list(a["OCC_CODE"]) == True:
            a.loc[a["OCC_CODE"] == my_list[i],"TOT_EMP2"] = list(b.loc[b["OCC_CODE"] == my_list[i],key])
        else:
            pass
    return a

concat_data(df1,df2,"TOT_EMP")

In [11]:
my_list = list(df2["OCC_CODE"])
for i in range(0,len(my_list)):
    if my_list[i] in list(df1["OCC_CODE"]):
        df1.loc[df1["OCC_CODE"] == my_list[i],"TOT_EMP2"] = list(df2.loc[df2["OCC_CODE"] == my_list[i],"TOT_EMP"])
df1.head()

## For some reason removing (== True) makes this work ????

,OCC_CODE,OCC_TITLE,TOT_EMP,TOT_EMP2
0,00-0000,All Occupations,146875480,135128260.0
1,11-0000,Management Occupations,8054120,6741640.0
2,11-1000,Top Executives,2658440,2351130.0
3,11-1010,Chief Executives,205890,246240.0
4,11-1020,General and Operations Managers,2400280,2049870.0


In [12]:
df1.dropna(inplace=True)
df1["CHANGE_EMP"] = df1["TOT_EMP"] - df1["TOT_EMP2"]
df1.head()

,OCC_CODE,OCC_TITLE,TOT_EMP,TOT_EMP2,CHANGE_EMP
0,00-0000,All Occupations,146875480,135128260.0,11747220.0
1,11-0000,Management Occupations,8054120,6741640.0,1312480.0
2,11-1000,Top Executives,2658440,2351130.0,307310.0
3,11-1010,Chief Executives,205890,246240.0,-40350.0
4,11-1020,General and Operations Managers,2400280,2049870.0,350410.0


In [23]:
#Now lets work on adding the skill values:

df3 = pd.read_csv("/Users/jacob/Documents/4YP data/ONET_18-10-21/test.csv")

In [24]:
def title_set(my_string):
    my_output = my_string[:-3]
    return my_output

df3.iloc[:,0] = df3.iloc[:,0].apply(title_set)
df3 = df3.drop_duplicates(subset = ["O*NET-SOC Code"],keep="first")
df3.reset_index(drop=True,inplace=True)
df3.head()
#title_set("11-1011.00")

,O*NET-SOC Code,Title,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,...,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources,Auto label value
0,11-1011,Chief Executives,4.75,4.88,4.38,4.88,3.62,1.12,4.75,4.75,...,0.0,1.00,5.75,5.38,5.12,4.75,5.50,4.75,5.38,0.0
1,11-1021,General and Operations Managers,4.00,4.00,3.88,4.00,2.50,1.12,4.00,3.62,...,0.0,2.12,3.75,3.00,3.12,3.75,3.38,3.25,3.88,NaN
2,11-2011,Advertising and Promotions Managers,4.00,4.12,3.88,4.12,3.25,0.62,4.12,4.12,...,0.0,1.00,4.00,3.12,3.75,3.88,3.62,2.62,3.88,NaN
3,11-2021,Marketing Managers,4.25,4.12,3.88,4.12,3.12,1.50,4.25,4.12,...,0.0,1.38,4.00,3.75,3.75,3.75,3.75,2.75,3.88,NaN
4,11-2022,Sales Managers,4.00,4.00,4.00,4.12,3.25,0.62,4.00,3.88,...,0.0,1.38,4.00,3.75,3.88,3.88,3.62,3.00,4.00,NaN


In [14]:
df1.head(30)

,OCC_CODE,OCC_TITLE,TOT_EMP,TOT_EMP2,CHANGE_EMP
0,00-0000,All Occupations,146875480,135128260.0,11747220.0
1,11-0000,Management Occupations,8054120,6741640.0,1312480.0
2,11-1000,Top Executives,2658440,2351130.0,307310.0
3,11-1010,Chief Executives,205890,246240.0,-40350.0
4,11-1020,General and Operations Managers,2400280,2049870.0,350410.0
5,11-1030,Legislators,52280,55020.0,-2740.0
6,11-2000,"Advertising, Marketing, Promotions, Public Rel...",770540,629670.0,140870.0
7,11-2010,Advertising and Promotions Managers,25100,29340.0,-4240.0
8,11-2020,Marketing and Sales Managers,666280,543410.0,122870.0
9,11-2021,Marketing Managers,263680,184490.0,79190.0


In [26]:
df3.head(30)

,O*NET-SOC Code,Title,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,...,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources,Auto label value
0,11-1011,Chief Executives,4.75,4.88,4.38,4.88,3.62,1.12,4.75,4.75,...,0.00,1.00,5.75,5.38,5.12,4.75,5.50,4.75,5.38,0.0
1,11-1021,General and Operations Managers,4.00,4.00,3.88,4.00,2.50,1.12,4.00,3.62,...,0.00,2.12,3.75,3.00,3.12,3.75,3.38,3.25,3.88,NaN
2,11-2011,Advertising and Promotions Managers,4.00,4.12,3.88,4.12,3.25,0.62,4.12,4.12,...,0.00,1.00,4.00,3.12,3.75,3.88,3.62,2.62,3.88,NaN
3,11-2021,Marketing Managers,4.25,4.12,3.88,4.12,3.12,1.50,4.25,4.12,...,0.00,1.38,4.00,3.75,3.75,3.75,3.75,2.75,3.88,NaN
4,11-2022,Sales Managers,4.00,4.00,4.00,4.12,3.25,0.62,4.00,3.88,...,0.00,1.38,4.00,3.75,3.88,3.88,3.62,3.00,4.00,NaN
5,11-3012,Administrative Services Managers,4.00,4.00,3.75,3.88,2.12,0.00,3.88,3.38,...,0.00,2.12,3.12,3.12,3.12,4.00,2.88,2.88,3.75,NaN
6,11-3021,Computer and Information Systems Managers,4.00,4.00,4.00,3.88,3.12,1.38,4.00,3.88,...,0.75,3.00,3.75,3.50,3.88,3.75,3.38,3.12,3.75,NaN
7,11-3031,Financial Managers,4.00,4.00,4.00,4.00,3.38,0.38,4.00,4.00,...,0.12,0.75,4.00,3.38,3.62,4.00,3.38,1.88,4.00,NaN
8,11-3051,Industrial Production Managers,4.00,4.00,4.00,4.00,3.75,1.12,4.00,4.00,...,1.62,3.88,4.38,3.88,3.75,4.12,4.00,4.00,4.25,NaN
9,11-3061,Purchasing Managers,3.75,4.00,4.00,3.88,3.38,0.25,4.00,3.88,...,0.00,1.88,3.75,3.62,3.62,3.88,3.88,3.75,4.00,NaN
